In [1]:
import pandas as pd
import torch
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import sys
import os
import pickle

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdmolops

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

RDKit WARNING: [17:41:43] Enabling RDKit 2019.09.3 jupyter extensions


### Load data
- Create variables for data folders
- Load data from excel
- Load data from non PBT excel and concatenate with PBT/vPvB column
- Create 3 datasets for PBT/CMR/PBT+

In [2]:
#data variables
data_folder = 'raw_data/'
data_source = '1 Supplemental Excel data.xlsx'
data_source_non = "Non-PBT.xlsx"

#robustnes and smile variables
robcmr = 'Excluded in robustness check 2 - CMR'
robpbt = 'Excluded in robustness check 2 - PBT/vPvB'
fp_cols = ['SMILES']

#cuda check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Load all data from excel supplementary information
chems_data  = pd.read_excel(data_folder+data_source, sheet_name = 1, header=1)

#Load PBT+ data
non_pbt = pd.read_excel(data_folder+data_source_non)
non_pbt = non_pbt['SMILES']
non_pbt = pd.concat([non_pbt, pd.DataFrame(np.zeros(non_pbt.size), columns=["PBT/vPvB"])], axis=1)


# CMR dataset
chems_ext = chems_data[fp_cols+['CMR']].loc[chems_data[robcmr] == 0]
# PBT dataset
chems_maccs = chems_data[fp_cols+['PBT/vPvB']].loc[chems_data[robpbt] == 0]
# PBT+ dataset
chems_maccs_non = pd.concat([chems_maccs, non_pbt], axis=0)



#pZZS dataset
chems_pzzs = pd.read_csv(data_folder+"pZZS_alles.csv")

In [81]:
#pZZS dataset
chems_pzzs = pd.read_csv(data_folder+"pZZS_alles.csv", names=["Label", "SMILES"])
chems_pzzs = chems_pzzs.iloc[1:]
chems_pzzs['Label'] = 1
print(chems_pzzs)

     Label                                             SMILES
1        1      Cc1ccc(N=Nc2c(O)ccc3ccccc23)c(c1)[N+]([O-])=O
2        1               CC1CC(C)(C)c2cc(C(C)=O)c(C)cc2C1(C)C
3        1      C[Si](C)(C)O[Si](C)(O[Si](C)(C)C)O[Si](C)(C)C
4        1                   C[SiH](O[Si](C)(C)C)O[Si](C)(C)C
5        1                                         [nH]1cncn1
..     ...                                                ...
127      1                       Oc1cc(Cl)ccc1Oc2ccc(Cl)cc2Cl
128      1                                    CO[Si](C)(OC)OC
129      1  CCCCC(CC)Cc1c(CC(CC)CCCC)c(C([O-])=O)c(C([O-])...
130      1                                COC([SiH3])=C(OC)OC
131      1               [Zn++].CN(C)C([S-])=S.CN(C)C([S-])=S

[131 rows x 2 columns]


### Load all SMARTS for parsing
- SMARTS from fingerprint and literature for 4 different characteristics
- SMARTS from CYPs from literature, loaded from excle data

In [4]:
#SMARTS for characteristics
Hphobe = "[C&!$(C=[O,N,P,S])&!$(C#N),c,s,S&H0&v2,F,Cl,Br,I]"

Acceptor = "[$([O,S;H1;v2;!$(*-*=[O,N,P,S])]),\
$([O,S;H0;v2]),\
$([O,S;-]),\
$([N;v3;!$(N-*=[O,N,P,S])]),\
n&H0&+0,\
$([o,s;+0;!$([o,s]:n);!$([o,s]:c:n)])]"

Donor="[$([N;!H0;v3,v4&+1]),\
$([O,S;H1;+0]),\
n&H1&+0]"

Aromatic =   "[a]"

#Feature list for parsing in functions
feature_list = [Acceptor, Donor, Aromatic, Hphobe]
label_list   = ['HB acceptor', 'HB donor', 'Aromatic', 'Hydrophobic']

#SOMs SMARTS from file
cyp_metabol_feats = pd.read_excel(data_folder + "output.xls")

### Function for all other functions

Creates 2 lists, one with all representations and with the dictionaries containings the mappings
- Rep 1 = Conventional
- Rep 2 = Hydro aromatic representation
- Rep 3 = Bonds representation
- Rep 4 = SOM representation

In [5]:
def create_dataset(SmilesData, Feats, Bonds, ToxLab):
    Rep1 = create_representation1(SmilesData, Feats, Bonds, ToxLab)
    Rep1, Rep1Dict = prepare_onehotencoded(Rep1)
    Rep1 = transform_DGL(Rep1)

    Rep2 = create_representation2(SmilesData, Feats, Bonds, ToxLab)
    Rep2, Rep2Dict = prepare_onehotencoded(Rep2)
    Rep2 = transform_DGL(Rep2)

    Rep3 = create_representation3(SmilesData, Feats, Bonds, ToxLab)
    Rep3, Rep3Dict = prepare_onehotencoded(Rep3)
    Rep3 = transform_DGL(Rep3)

    Rep4 = create_representation4(SmilesData, Feats, Bonds, ToxLab)
    Rep4, Rep4Dict = prepare_onehotencoded(Rep4)
    Rep4 = transform_DGL(Rep4)
    
    return [Rep1, Rep2, Rep3, Rep4], [Rep1Dict, Rep2Dict, Rep3Dict, Rep4Dict]

### Function to create the conventional representation

- Parse the molecules, load the molecule from the smiles and get the information required.
- Parse the atomic numbers of the atoms, parse the bond types, both for the labels.
- Convert into required data formatting.


In [6]:
def create_representation1(smiles_data, features, dictionary_bonds, tox_label):
    matrices_dataset = []
    
    for mol_idx in range(smiles_data.shape[0]):
        #get molecule from SMILES
        mcule = (Chem.MolFromSmiles(smiles_data['SMILES'].iloc[mol_idx]))
        #adjacency matrix, molecule in graph and edge list
        adj_matrix = Chem.GetAdjacencyMatrix(mcule)
        mcule_graph = nx.from_numpy_matrix(adj_matrix)
        edge_list = (np.tril(adj_matrix) != 0).nonzero()
        
        #get label for each atom as list
        atom_labels = []
        for _, atom in enumerate(mcule.GetAtoms()):
            atom_labels.append(atom.GetAtomicNum())
        
        #add edge label as weight to the graph representation, use dict of bonds
        for atom1,atom2 in np.transpose(edge_list):
            bond_str = str(mcule.GetBondBetweenAtoms(int(atom1),int(atom2)).GetBondType())
            mcule_graph[atom1][atom2]['weight'] = dictionary_bonds.get(bond_str)

        #add adjancency matrix with edge labels, atom labels and label to dataset
        adj_mat = nx.to_numpy_matrix(mcule_graph).astype(int)
        tox_lab = smiles_data[tox_label].iloc[mol_idx]
        matrices_dataset.append([adj_mat, atom_labels, tox_lab])


    return pd.DataFrame(matrices_dataset, columns=['matrix', 'atoms', 'tox'])

### Function to create the Hydro/Aromatic representation

- Parse the molecules, load the molecule from the smiles and get the information required.
- Parse the atomic numbers of the atoms, parse the bond types, both for the labels.
- Parse the SMARTS from the features and add as nodes with the labels.
- Convert into required data formatting.


In [7]:
def create_representation2(smiles_data, features, dictionary_bonds, tox_label):
    matrices_dataset = []
    
    for mol_idx in range(smiles_data.shape[0]):
        #get molecule from SMILES and feature locations list
        mcule = Chem.MolFromSmiles(smiles_data['SMILES'].iloc[mol_idx])
        feat1 = mcule.GetSubstructMatches(Chem.MolFromSmarts(features[0]))
        feat2 = mcule.GetSubstructMatches(Chem.MolFromSmarts(features[1]))
        feat3 = mcule.GetSubstructMatches(Chem.MolFromSmarts(features[2]))
        feat4 = mcule.GetSubstructMatches(Chem.MolFromSmarts(features[3]))
        
        #adjacency matrix, molecule in graph and edge list
        adj_matrix = Chem.GetAdjacencyMatrix(mcule)
        mcule_graph = nx.from_numpy_matrix(adj_matrix)
        edge_list = (np.tril(adj_matrix) != 0).nonzero()
        
        #get label for each atom as list
        atom_labels = []
        for _, atom in enumerate(mcule.GetAtoms()):
            atom_labels.append(atom.GetAtomicNum())
        
        #add edge label as weight to the graph representation, use dict of bonds
        for atom1,atom2 in np.transpose(edge_list):
            bond_str = str(mcule.GetBondBetweenAtoms(int(atom1),int(atom2)).GetBondType())
            mcule_graph[atom1][atom2]['weight'] = dictionary_bonds.get(bond_str)

        #get start of idx for new nodes
        node_count = len(mcule_graph.nodes)
        
        #add all new features as nodes
        for idx, feat_list in enumerate([feat1,feat2,feat3,feat4]):
#         for idx, feat_list in enumerate([feat1,feat2]):
            for feat in feat_list:
                mcule_graph.add_node(node_count)
                mcule_graph.add_edge(feat[0], node_count, weight=(len(dictionary_bonds)+1+idx))
                #add the new nodes as numbers above the current periodic table
                atom_labels.append(120+idx)
                node_count+=1
        
        #add adjancency matrix with edge labels, atom labels and label to dataset
        adj_mat = nx.to_numpy_matrix(mcule_graph).astype(int)
        tox_lab = smiles_data[tox_label].iloc[mol_idx]
        matrices_dataset.append([adj_mat, atom_labels, tox_lab])


    return pd.DataFrame(matrices_dataset, columns=['matrix', 'atoms', 'tox'])

### Function to create the Bonds representation

- Parse the molecules, load the molecule from the smiles and get the information required.
- Parse the atomic numbers of the atoms, parse the bond types, both for the labels.
- Parse the bonds and add as nodes with the labels.
- Convert into required data formatting.

In [8]:
def create_representation3(smiles_data, features, dictionary_bonds, tox_label):
    matrices_dataset = []
    
    for mol_idx in range(smiles_data.shape[0]):
        #get molecule from SMILES
        mcule = Chem.MolFromSmiles(smiles_data['SMILES'].iloc[mol_idx])
        
        #adjacency matrix, molecule in graph and edge list
        adj_matrix = Chem.GetAdjacencyMatrix(mcule)
        mcule_graph = nx.from_numpy_matrix(adj_matrix)
        edge_list = (np.tril(adj_matrix) != 0).nonzero()

        #get label for each atom as list
        atom_labels = []
        for _, atom in enumerate(mcule.GetAtoms()):
            atom_labels.append(atom.GetAtomicNum())
        
        #get start of idx for new nodes
        node_count = len(mcule_graph.nodes)
        
        #add edge label as weight to the graph representation, use dict of bonds
        for atom1,atom2 in np.transpose(edge_list):
            bond_str = str(mcule.GetBondBetweenAtoms(int(atom1),int(atom2)).GetBondType())
            mcule_graph.remove_edge(atom1,atom2)
            mcule_graph.add_node(node_count)
            mcule_graph.add_edge(atom1, node_count, weight=dictionary_bonds.get(bond_str))
            mcule_graph.add_edge(atom2, node_count, weight=dictionary_bonds.get(bond_str))
            atom_labels.append(120+dictionary_bonds.get(bond_str))
            node_count += 1


        #add adjancency matrix with edge labels, atom labels and label to dataset
        adj_mat = nx.to_numpy_matrix(mcule_graph).astype(int)
        tox_lab = smiles_data[tox_label].iloc[mol_idx]
        matrices_dataset.append([adj_mat, atom_labels, tox_lab])


    return pd.DataFrame(matrices_dataset, columns=['matrix', 'atoms', 'tox'])

### Function to create the SOMs representation

- Parse the molecules, load the molecule from the smiles and get the information required.
- Parse the atomic numbers of the atoms, parse the bond types, both for the labels.
- Parse the SMARTS from the CYPs and add as nodes with the labels.
- Convert into required data formatting.

In [9]:
def create_representation4(smiles_data, features, dictionary_bonds, tox_label):
    matrices_dataset = []
    
    for mol_idx in range(smiles_data.shape[0]):
        #get molecule from SMILES
        mcule = (Chem.MolFromSmiles(smiles_data['SMILES'].iloc[mol_idx]))
        
        #adjacency matrix, molecule in graph and edge list
        adj_matrix = Chem.GetAdjacencyMatrix(mcule)
        mcule_graph = nx.from_numpy_matrix(adj_matrix)
        edge_list = (np.tril(adj_matrix) != 0).nonzero()
        
        #get label for each atom as list
        atom_labels = []
        for _, atom in enumerate(mcule.GetAtoms()):
            atom_labels.append(atom.GetAtomicNum())
        
        #add edge label as weight to the graph representation, use dict of bonds
        for atom1,atom2 in np.transpose(edge_list):
            bond_str = str(mcule.GetBondBetweenAtoms(int(atom1),int(atom2)).GetBondType())
            mcule_graph[atom1][atom2]['weight'] = dictionary_bonds.get(bond_str)
        
        #Create matrix with multi hot encoded array for every atom and metabolism features
        feat_cyp = np.zeros((len(list(mcule.GetAtoms())), 71))
        for idx in range(cyp_metabol_feats.shape[0]):
            cyp_idx_feats = mcule.GetSubstructMatches(Chem.MolFromSmarts(cyp_metabol_feats['SMARTS'].iloc[idx]))
            for feat in cyp_idx_feats:
                feat_cyp[feat[0]][idx] = 1
        
        #add adjancency matrix with edge labels, atom labels and label to dataset
        adj_mat = nx.to_numpy_matrix(mcule_graph).astype(int)
        tox_lab = smiles_data[tox_label].iloc[mol_idx]
        matrices_dataset.append([adj_mat, atom_labels, tox_lab, feat_cyp])

    return pd.DataFrame(matrices_dataset, columns=['matrix', 'atoms', 'tox', 'cyps'])

### Function to prepare labels to one hot labels

- Parse label list to range higher than possible
- Parse label list to 0 to n mapping

In [10]:
def prepare_onehotencoded(rep_dataset):
    atom_list = [x for x in rep_dataset['atoms'].values]
    atom_flat = [item for sublist in atom_list for item in sublist]
    atom_set  = sorted(list(set(atom_flat)))
    
    #first replace with numbers above max of atom set to avoid multiple wrong replacing
    replace1  = list(range(max(atom_set)+1, len(atom_set)+max(atom_set)+1))
    for i, a_list in enumerate(atom_list):
        for j, atom_l in enumerate(a_list):
            for idx, atom_s in enumerate(atom_set):
                if atom_l == atom_s:
                    atom_list[i][j] = replace1[idx]
                    
    #second replace with numbers in range zero to length atom set 
    replace2  = list(range(len(replace1)))
    for i, a_list in enumerate(atom_list):
        for j, atom_l in enumerate(a_list):
            for idx, atom_s in enumerate(replace1):
                if atom_l == atom_s:
                    atom_list[i][j] = replace2[idx]
    
    rep_dataset['atoms'] = atom_list
    dict_onehot = {}
    for i, d in enumerate(replace2):
        dict_onehot[d] = atom_set[i]

    return rep_dataset, dict_onehot

### Function to required data formatting
- Create list with atom type feature vectors
- Bond type adjacency matrix with labels as indices
- Label for the tox

In [11]:
def transform_DGL(dataframe):
    DGL_list = []
    #convert to correct conventions
    max_atoms = max([max(atoms) for atoms in dataframe['atoms']])
    dataframe['matrix'] = dataframe['matrix'].apply(lambda x: torch.tensor(x))
    dataframe['atoms']  = dataframe['atoms'].apply(lambda x: torch.tensor(x))
    #create list of graph info dictionaries
    for i in range(dataframe.shape[0]):
        mol_dict = {}
        mol_dict['num_atom']  = int(len(dataframe['atoms'].iloc[i]))
        if 'cyps' in dataframe.columns:
            onehot_atoms = np.eye(max_atoms+1)[dataframe['atoms'].iloc[i]]
            onehot_cyps = dataframe['cyps'].iloc[i]
            mol_dict['atom_type'] = np.concatenate((onehot_atoms,onehot_cyps), axis=1)
        else:
            mol_dict['atom_type'] = np.eye(max_atoms+1)[dataframe['atoms'].iloc[i]]
        mol_dict['bond_type'] = dataframe['matrix'].iloc[i]
        mol_dict['label']     = dataframe['tox'].iloc[i]
        DGL_list.append(mol_dict)
    return DGL_list

### Function to save the representations to the data folder
Save the dictionary mappings and the data for every representation

In [12]:
def save_reps(reps, dicts, save_name):
    for idx, rep in enumerate(reps):
        pickle.dump(rep, open(save_name + str(idx+1) + ".pickle","wb"))
        pickle.dump(dicts[idx], open(save_name + str(idx+1) + "_dict.pickle","wb"))

In [13]:
#remove not parseable molecules
# these molecules could not be parse with RDKIT and gave an error
if chems_maccs_non.shape[0] == 975:
    chems_maccs_non = chems_maccs_non.drop(chems_maccs_non.index[506])
    chems_maccs_non = chems_maccs_non.drop(chems_maccs_non.index[751])
    chems_maccs_non = chems_maccs_non.drop(chems_maccs_non.index[794])

### Parse representations and save them
Call the above functions

In [14]:
bond_dict = {'SINGLE':1, 'DOUBLE':2, 'TRIPLE':3, 'AROMATIC':4}

CMR_reps, CMR_dicts = create_dataset(chems_ext, feature_list, bond_dict, 'CMR')
PBT_reps, PBT_dicts = create_dataset(chems_maccs, feature_list, bond_dict, 'PBT/vPvB')
PBTn_reps, PBTn_dicts = create_dataset(chems_maccs_non, feature_list, bond_dict, 'PBT/vPvB')


In [15]:
save_reps(CMR_reps, CMR_dicts, "data/CMR_Rep")
save_reps(PBT_reps, PBT_dicts, "data/PBT_Rep")
save_reps(PBTn_reps, PBTn_dicts, "data/PBTn_Rep")

In [85]:
bond_dict = {'SINGLE':1, 'DOUBLE':2, 'TRIPLE':3, 'AROMATIC':4}

pZZS_reps, pZZS_dicts = create_dataset(chems_pzzs, feature_list, bond_dict, "Label")
save_reps(pZZS_reps, pZZS_dicts, "data/pZZS_Rep")

In [86]:
print(pZZS_dicts[2].values())

dict_values([6, 7, 8, 9, 14, 15, 16, 17, 30, 35, 51, 121, 122, 123, 124])


In [87]:
print(PBTn_dicts[2].values())

dict_values([6, 7, 8, 9, 14, 15, 16, 17, 35, 50, 53, 121, 122, 123, 124])


In [83]:
for mol_idx in range(chems_pzzs.shape[0]):
    

    mcule = Chem.MolFromSmiles(chems_pzzs['SMILES'].iloc[mol_idx])
    atom_labels=[]
    for _, atom in enumerate(mcule.GetAtoms()):
        atom_labels.append(atom.GetAtomicNum())
    if 20 in atom_labels:
        print(mol_idx, 20)
    if 29 in atom_labels:
        print(mol_idx, 29)
    if 30 in atom_labels:
        print(mol_idx, 30)
    if 51 in atom_labels:
        print(mol_idx, 51)
    if 51 in atom_labels:
        print(mol_idx, 51)
    if 51 in atom_labels:
        print(mol_idx, 51)

31 51
31 51
31 51
128 30


In [82]:
chems_pzzs = chems_pzzs.drop(chems_pzzs.index[75])
chems_pzzs = chems_pzzs.drop(chems_pzzs.index[72])

In [62]:
print(chems_pzzs.shape)

(129, 2)
